<a href="https://colab.research.google.com/github/swazypat/Kaggle_Projects/blob/main/Fpl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet pandas numpy aiohttp fpl understatapi fuzzywuzzy unidecode matplotlib nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.0/244.0 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import asyncio
import logging
import warnings
import nest_asyncio
import aiohttp
import requests
from understatapi import UnderstatClient
from fpl import FPL
from fuzzywuzzy import process as fuzzy_process
import unidecode

nest_asyncio.apply()
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
class Config:
    DEFAULT_LEAGUE_UNDERSTAT = "EPL"
    DEFAULT_SEASON_START_YEAR = 2024
    RECENT_FORM_WINDOW_MATCHES = 6
    MIN_MINUTES_DISPLAY = 270
    NEXT_GWS_TO_CONSIDER = 5
    MIN_FPL_PRICE = 4.0
    MAX_FPL_PRICE = 15.0
    AIOHTTP_TIMEOUT_SECONDS = 30
    FUZZY_MATCH_SCORE_THRESHOLD = 82
    WEIGHTS = {
        'xg_xa_p90': 0.30,
        'recent_slope': 0.10,
        'fpl_points_p_gw_recent': 0.25,
        'fpl_points_p_mill': 0.15,
        'upcoming_fdr': -0.20,
    }
    TEAM_NAME_MAP = {
        "Manchester City": "Man City", "Manchester United": "Man Utd", "Newcastle United": "Newcastle",
        "Nottingham Forest": "Nott'm Forest", "Tottenham": "Spurs", "West Ham": "West Ham",
        "Wolverhampton Wanderers": "Wolves", "Brighton": "Brighton", "Crystal Palace": "Crystal Palace",
        "Leicester": "Leicester", "Sheffield United": "Sheffield Utd", "Aston Villa": "Aston Villa",
        "Bournemouth": "Bournemouth", "Fulham": "Fulham", "Liverpool": "Liverpool", "Arsenal": "Arsenal",
        "Brentford": "Brentford", "Chelsea": "Chelsea", "Everton": "Everton", "Ipswich": "Ipswich",
        "Southampton": "Southampton",
    }
    REVERSE_TEAM_NAME_MAP = {v: k for k, v in TEAM_NAME_MAP.items()}
    MANUAL_NAME_MAP = {
        "Marcus Rashford": "Marcus Rashford", "Bernardo Silva": "Bernardo Fernandes da Silva Junior",
        "Rodri": "Rodrigo Hernandez", "Pedro Neto": "Pedro Lomba Neto", "João Gomes": "Joao Gomes",
        "Tomas Soucek": "Tomas Soucek", "Matheus Cunha": "Matheus Santos Carneiro Da Cunha",
        "Nélson Semedo": "Nelson Cabral Semedo", "Bobby Reid": "Bobby De Cordova-Reid",
        "Emerson": "Emerson Palmieri dos Santos", "Mateo Kovacic": "Mateo Kovacic",
        "Trevoh Chalobah": "Trevoh Chalobah", "Axel Disasi": "Axel Disasi",
        "Carlos Alcaraz": "Carlos Alcaraz", "Neto": "Norberto Murara Neto"
    }

def normalize_name(name):
    if not isinstance(name, str): return ""
    return unidecode.unidecode(name).lower()

def calculate_slope(series):
    y = series.values; x = np.arange(len(y))
    if len(x) < 2: return 0.0
    try: coeffs = np.polyfit(x, y, 1); return coeffs[0] if np.all(np.isfinite(coeffs)) else 0.0
    except Exception: return 0.0

In [5]:

def get_player_understat_data(understat, player_id, season_start_year, recent_form_matches=Config.RECENT_FORM_WINDOW_MATCHES):
    # Synchronous - for thread/executor, as UnderstatAPI is sync
    try:
        player_endpoint = understat.player(player=player_id)
        matches = player_endpoint.get_match_data()
        if not matches: return None
        df = pd.DataFrame(matches)
        for col in ['date', 'xG', 'xA', 'time']:
            if col not in df.columns: return None
            if col != 'date': df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df.dropna(subset=['date'])
        if df.empty: return None
        df['xG+xA'] = df['xG'] + df['xA']
        season_start_dt = datetime(season_start_year, 7, 1)
        season_end_dt = datetime(season_start_year + 1, 6, 30)
        df_season = df[(df['date'] >= season_start_dt) & (df['date'] <= season_end_dt)].copy()
        if df_season.empty: return None
        df_season = df_season.sort_values('date', ascending=True).reset_index(drop=True)
        total_minutes_played = df_season['time'].sum()
        if total_minutes_played < 1: return None
        total_xg = df_season['xG'].sum()
        total_xa = df_season['xA'].sum()
        total_xg_xa = df_season['xG+xA'].sum()
        volatility_combined = df_season['xG+xA'].std() if len(df_season) >= 2 else 0
        window_recent = min(recent_form_matches, len(df_season))
        recent_matches_df = df_season.tail(window_recent).copy()
        recent_avg_combined = recent_matches_df['xG+xA'].mean() if not recent_matches_df.empty else 0
        recent_slope_combined = calculate_slope(recent_matches_df['xG+xA']) if not recent_matches_df.empty else 0
        xg_p90 = (total_xg / total_minutes_played) * 90 if total_minutes_played > 0 else 0
        xa_p90 = (total_xa / total_minutes_played) * 90 if total_minutes_played > 0 else 0
        xg_xa_p90 = (total_xg_xa / total_minutes_played) * 90 if total_minutes_played > 0 else 0
        return {
            'understat_id': player_id,
            'Total_Minutes_Played_understat': total_minutes_played,
            'xG_p90': xg_p90, 'xA_p90': xa_p90, 'xG_xA_p90': xg_xa_p90,
            'Recent_Slope_Combined': recent_slope_combined,
            'Volatility_Combined': volatility_combined,
            'Recent_Understat_Matches': len(recent_matches_df)
        }
    except Exception as e:
        logging.warning(f"[Understat fetch error] {e}")
        return None

async def fetch_all_understat_data(loop, season_start_year):
    print("Fetching Understat summary...")
    understat = UnderstatClient()
    try:
        players_summary = understat.league(league=Config.DEFAULT_LEAGUE_UNDERSTAT).get_player_data(season=str(season_start_year))
    except Exception as e:
        print("ERROR fetching Understat summary:", e); return pd.DataFrame(), {}
    summary_lookup = {p['id']: {'name': p.get('player_name'), 'team': p.get('team_title')} for p in players_summary if p.get('id')}
    tasks = [loop.run_in_executor(None, get_player_understat_data, understat, p['id'], season_start_year) for p in players_summary if p.get('id')]
    understat_results = []
    for idx, task in enumerate(asyncio.as_completed(tasks), 1):
        try: result = await task
        except Exception as e: result = None
        if result: understat_results.append(result)
        if idx % 20 == 0 or idx == len(tasks):
            print(f"\rProcessed {idx}/{len(tasks)} Understat...", end="")
    print()
    if not understat_results:
        print("WARNING: No Understat details processed.")
        return pd.DataFrame(), summary_lookup
    df_understat = pd.DataFrame(understat_results)
    df_understat['understat_name'] = df_understat['understat_id'].map(lambda x: summary_lookup.get(x, {}).get('name'))
    df_understat['understat_team_original'] = df_understat['understat_id'].map(lambda x: summary_lookup.get(x, {}).get('team'))
    df_understat.dropna(subset=['understat_name', 'understat_team_original'], inplace=True)
    def clean_team(team_str):
        return team_str.split(',')[-1].strip() if isinstance(team_str, str) and ',' in team_str else team_str
    df_understat['understat_team'] = df_understat['understat_team_original'].apply(clean_team)
    df_understat['understat_norm_name'] = df_understat['understat_name'].apply(normalize_name)
    return df_understat, summary_lookup

In [6]:

async def fetch_fpl_data():
    print("Fetching FPL data...")
    session = None
    try:
        timeout = aiohttp.ClientTimeout(total=Config.AIOHTTP_TIMEOUT_SECONDS)
        session = aiohttp.ClientSession(timeout=timeout)
        fpl = FPL(session)
        players_data, teams_data, events_data, fixtures_data = await asyncio.gather(
            fpl.get_players(return_json=True),
            fpl.get_teams(return_json=True),
            fpl.get_gameweeks(return_json=True),
            fpl.get_fixtures(return_json=True)
        )
        await session.close()
        if not players_data or not teams_data or not events_data or not fixtures_data:
            raise ValueError("FPL API returned empty data.")
        return (pd.DataFrame(players_data), pd.DataFrame(teams_data), pd.DataFrame(events_data), pd.DataFrame(fixtures_data))
    except Exception as e:
        print("ERROR fetching FPL data:", e)
        if session and not session.closed: await session.close()
        return None, None, None, None

In [7]:

def merge_data(df_understat, understat_lookup, df_fpl_players, df_fpl_teams):
    if df_understat.empty or df_fpl_players.empty:
        print("Can't merge, empty Understat or FPL df.")
        return pd.DataFrame(), []
    fpl_team_lookup = df_fpl_teams.set_index('id')['name'].to_dict()
    df_fpl_players['fpl_team_name'] = df_fpl_players['team'].map(fpl_team_lookup)
    df_fpl_players['fpl_web_name'] = df_fpl_players['web_name']
    df_fpl_players['fpl_full_name'] = df_fpl_players['first_name'] + " " + df_fpl_players['second_name']
    df_fpl_players['fpl_norm_full_name'] = df_fpl_players['fpl_full_name'].apply(normalize_name)
    df_fpl_players['fpl_norm_web_name'] = df_fpl_players['fpl_web_name'].apply(normalize_name)
    df_fpl_players['fpl_position_id'] = pd.to_numeric(df_fpl_players['element_type'], errors='coerce').fillna(0).astype(int)
    pos_map = {1: "Goalkeeper", 2: "Defender", 3: "Midfielder", 4: "Forward"}
    df_fpl_players['fpl_position'] = df_fpl_players['fpl_position_id'].map(pos_map).fillna("Unknown")
    df_fpl_players['fpl_price'] = df_fpl_players['now_cost'] / 10.0
    fpl_full_groups = df_fpl_players.groupby('fpl_norm_full_name')
    fpl_web_groups = df_fpl_players.groupby('fpl_norm_web_name')
    merged_data = []
    unmatched_understat = []
    norm_full = set(df_fpl_players['fpl_norm_full_name'])
    norm_web = set(df_fpl_players['fpl_norm_web_name'])
    for _, u_row in df_understat.iterrows():
        u_name = u_row['understat_name']
        u_norm_name = u_row['understat_norm_name']
        u_team = u_row['understat_team']
        # Manual map
        manual_target = Config.MANUAL_NAME_MAP.get(u_name)
        potential_fpl_players = []
        if manual_target:
            norm_manual = normalize_name(manual_target)
            if norm_manual in norm_full:
                group = fpl_full_groups.get_group(norm_manual)
                potential_fpl_players = group.to_dict('records')
        # Fuzzy
        if not potential_fpl_players:
            match = fuzzy_process.extractOne(u_norm_name, list(norm_full), score_cutoff=Config.FUZZY_MATCH_SCORE_THRESHOLD)
            if match:
                group = fpl_full_groups.get_group(match[0])
                potential_fpl_players = group.to_dict('records')
            else:
                match_web = fuzzy_process.extractOne(u_norm_name, list(norm_web), score_cutoff=Config.FUZZY_MATCH_SCORE_THRESHOLD)
                if match_web:
                    group = fpl_web_groups.get_group(match_web[0])
                    potential_fpl_players = group.to_dict('records')
        found = None
        for cand in potential_fpl_players:
            fpl_team = cand.get('fpl_team_name')
            mapped_u_team = Config.TEAM_NAME_MAP.get(u_team, u_team)
            if (u_team == fpl_team or mapped_u_team == fpl_team or u_team == Config.REVERSE_TEAM_NAME_MAP.get(fpl_team)):
                found = cand
                break
        if found is None and potential_fpl_players:
            found = potential_fpl_players[0]
        if found:
            merged_row = u_row.to_dict()
            fpl_cand = found.copy()
            merged_row['fpl_id'] = fpl_cand.pop('id', None)
            merged_row.update(fpl_cand)
            merged_row['fpl_player_name'] = fpl_cand.get('fpl_web_name')
            merged_row['fpl_full_name'] = fpl_cand.get('fpl_full_name')
            merged_data.append(merged_row)
        else:
            unmatched_understat.append(f"{u_name} ({u_team})")
    print(f"Matched {len(merged_data)}/{len(df_understat)} Understat players.")
    if unmatched_understat:
        print(f"Unmatched Understat players (first 10): {unmatched_understat[:10]}")
    return pd.DataFrame(merged_data), unmatched_understat

In [8]:
def get_upcoming_fdr(team_id, current_gw, fixtures_df, teams_df, num_gws):
    try:
        relevant_fixtures = fixtures_df[
            (fixtures_df['event'] > current_gw) &
            (fixtures_df['event'] <= current_gw + num_gws) &
            fixtures_df['event'].notna() &
            ((fixtures_df['team_h'] == team_id) | (fixtures_df['team_a'] == team_id))
        ].sort_values(by='event').copy()
        fdr_scores, fixture_details = [], []
        for _, row in relevant_fixtures.iterrows():
            gw = int(row['event'])
            is_home = row['team_h'] == team_id
            difficulty = row['team_h_difficulty'] if is_home else row['team_a_difficulty']
            opponent_id = row['team_a'] if is_home else row['team_h']
            opponent_info = teams_df.loc[teams_df['id'] == opponent_id]
            opponent_name = opponent_info['short_name'].iloc[0] if not opponent_info.empty else "UNK"
            venue = "(H)" if is_home else "(A)"
            if pd.notna(difficulty):
                fdr_scores.append(float(difficulty))
                fixture_details.append(f"GW{gw}:{opponent_name}{venue}({int(difficulty)})")
        avg_fdr = np.mean(fdr_scores) if fdr_scores else 5.0
        return avg_fdr, fixture_details
    except Exception as e:
        return 5.0, ["FDR Calc Error"]

def calculate_fpl_score(row, weights):
    try:
        xg_xa = row.get('xG_xA_p90', 0) if pd.notna(row.get('xG_xA_p90', 0)) else 0
        slope = row.get('Recent_Slope_Combined', 0) if pd.notna(row.get('Recent_Slope_Combined', 0)) else 0
        pts_recent = row.get('fpl_points_p_gw_recent', 0) if pd.notna(row.get('fpl_points_p_gw_recent', 0)) else 0
        pts_mill = row.get('fpl_points_p_mill', 0) if pd.notna(row.get('fpl_points_p_mill', 0)) else 0
        fdr = row.get('avg_upcoming_fdr', 5.0) if pd.notna(row.get('avg_upcoming_fdr', 5.0)) else 5.0
        fdr = max(1.0, min(5.0, fdr))
        fdr_score_component = (5.0 - fdr) / 4.0
        score = (
            weights['xg_xa_p90'] * xg_xa +
            weights['recent_slope'] * slope * 10 +
            weights['fpl_points_p_gw_recent'] * pts_recent +
            weights['fpl_points_p_mill'] * pts_mill +
            weights['upcoming_fdr'] * fdr_score_component
        )
        return float(score)
    except Exception: return -999.0

In [9]:

async def notebook_main():
    now_utc = datetime.now(timezone.utc)
    print("---- FPL/Understat notebook analyzer ----")
    print("Fetching FPL data...")
    players_df, teams_df, events_df, fixtures_df = await fetch_fpl_data()
    if any(x is None for x in (players_df, teams_df, events_df, fixtures_df)):
        print("ERROR: FPL data not loaded."); return
    events_df['deadline_time_dt'] = pd.to_datetime(events_df['deadline_time'])
    upcoming_gws = events_df[events_df['deadline_time_dt'] > now_utc].sort_values('id')
    if not upcoming_gws.empty:
        next_gw = int(upcoming_gws.iloc[0]['id'])
        current_gw = next_gw - 1
    else:
        current_gw = events_df['id'].max() if not events_df.empty else 0
        next_gw = current_gw + 1
    print(f"Current GW={current_gw}  Next={next_gw}")
    loop = asyncio.get_running_loop()
    df_understat, understat_lookup = await fetch_all_understat_data(loop, Config.DEFAULT_SEASON_START_YEAR)
    if df_understat is None or df_understat.empty:
        print("No Understat data, aborting."); return
    df_merged, unmatched = merge_data(df_understat, understat_lookup, players_df, teams_df)
    if df_merged is None or df_merged.empty:
        print("Merging failed, aborting."); return

    # ---- Fix for team id column missing ----
    if 'fpl_team_id' not in df_merged.columns:
        if 'team' in df_merged.columns:
            df_merged['fpl_team_id'] = df_merged['team']
        elif 'fpl_team' in df_merged.columns:
            df_merged['fpl_team_id'] = df_merged['fpl_team']
        else:
            print("ERROR: No team id col found!"); return
    # -----------------------------------------

    # ---- Fix for fpl_status column missing ----
    if 'fpl_status' not in df_merged.columns:
        if 'status' in df_merged.columns:  # most likely!
            df_merged['fpl_status'] = df_merged['status']
        else:
            print("ERROR: No fpl_status/status column found!")
            print("Available columns:", df_merged.columns.tolist())
            return
    # -------------------------------------------

    # FPL metrics
    df_merged['fpl_price'] = pd.to_numeric(df_merged['fpl_price'], errors='coerce').fillna(0)
    df_merged['total_points'] = pd.to_numeric(df_merged['total_points'], errors='coerce').fillna(0)
    df_merged['fpl_points_p_mill'] = np.where(
        df_merged['fpl_price'] > 0, df_merged['total_points'] / df_merged['fpl_price'], 0.0)
    df_merged['fpl_points_p_gw_recent'] = pd.to_numeric(df_merged['form'], errors='coerce').fillna(0)
    df_merged['fpl_team_id'] = pd.to_numeric(df_merged['fpl_team_id'], errors='coerce')
    df_merged.dropna(subset=['fpl_team_id'], inplace=True)
    df_merged['fpl_team_id'] = df_merged['fpl_team_id'].astype(int)
    fixtures_info = df_merged.apply(
        lambda row: get_upcoming_fdr(row['fpl_team_id'], current_gw, fixtures_df, teams_df, Config.NEXT_GWS_TO_CONSIDER),
        axis=1)
    df_merged['avg_upcoming_fdr'] = [info[0] for info in fixtures_info]
    df_merged['upcoming_fixtures_str'] = [", ".join(info[1]) for info in fixtures_info]
    df_merged['fpl_score'] = df_merged.apply(lambda row: calculate_fpl_score(row, Config.WEIGHTS), axis=1)
    # Filter, sort, display
    df_merged['Total_Minutes_Played_understat'] = pd.to_numeric(
        df_merged['Total_Minutes_Played_understat'], errors='coerce').fillna(0)
    df_display = df_merged[
        (df_merged['fpl_status'] == 'a') &
        (df_merged['Total_Minutes_Played_understat'] >= Config.MIN_MINUTES_DISPLAY) &
        (df_merged['fpl_price'] >= Config.MIN_FPL_PRICE) &
        (df_merged['fpl_price'] <= Config.MAX_FPL_PRICE)
    ].copy()
    display_cols = [
        'fpl_player_name', 'fpl_team_name', 'fpl_position', 'fpl_price',
        'fpl_score', 'total_points', 'fpl_points_p_mill', 'fpl_points_p_gw_recent',
        'xG_xA_p90', 'Recent_Slope_Combined', 'avg_upcoming_fdr',
        'upcoming_fixtures_str'
    ]
    df_sorted = df_display.sort_values('fpl_score', ascending=False).reset_index(drop=True)
    print(f"\n--- Top 20 FPL Players (GW{next_gw} onwards) ---\n")
    from IPython.display import display
    display(df_sorted[display_cols].head(20))
    # Optionally: save as CSV file
    df_sorted[display_cols].to_csv(f"FPL_Rankings_GW{next_gw}_{datetime.now().strftime('%Y%m%d_%H%M')}.csv", index=False)

# Run it!
await notebook_main()

---- FPL/Understat notebook analyzer ----
Fetching FPL data...
Fetching FPL data...
Current GW=32  Next=33
Fetching Understat summary...
Processed 550/550 Understat...
Matched 550/550 Understat players.

--- Top 20 FPL Players (GW33 onwards) ---



,fpl_player_name,fpl_team_name,fpl_position,fpl_price,fpl_score,total_points,fpl_points_p_mill,fpl_points_p_gw_recent,xG_xA_p90,Recent_Slope_Combined,avg_upcoming_fdr,upcoming_fixtures_str
0,Rogers,Aston Villa,Midfielder,5.6,5.891459,138,24.642857,8.3,0.448577,0.075457,3.2,"GW33:NEW(H)(3), GW33:MCI(A)(4), GW35:FUL(H)(3)..."
1,Strand Larsen,Wolves,Forward,5.4,5.751379,121,22.407407,8.5,0.451992,0.229670,3.0,"GW33:MUN(A)(3), GW34:LEI(H)(2), GW35:MCI(A)(4)..."
2,Elanga,Nott'm Forest,Midfielder,5.4,5.440651,131,24.259259,7.2,0.445875,-0.002001,2.4,"GW33:TOT(A)(3), GW34:BRE(H)(2), GW35:CRY(A)(3)..."
3,Luis Díaz,Liverpool,Midfielder,7.5,5.300922,160,21.333333,7.3,0.643513,0.172868,3.2,"GW33:LEI(A)(2), GW34:TOT(H)(3), GW35:CHE(A)(4)..."
4,J.Murphy,Newcastle,Midfielder,5.2,5.068678,122,23.461538,5.2,0.614627,0.155059,3.2,"GW33:AVL(A)(3), GW34:IPS(H)(2), GW35:BHA(A)(3)..."
5,Wissa,Brentford,Forward,6.5,5.001367,148,22.769231,6.0,0.679319,-0.017814,3.0,"GW33:BHA(H)(3), GW34:NFO(A)(4), GW35:MUN(H)(3)..."
6,Milenković,Nott'm Forest,Defender,5.1,4.946871,129,25.294118,5.0,0.159396,-0.015065,2.4,"GW33:TOT(A)(3), GW34:BRE(H)(2), GW35:CRY(A)(3)..."
7,Sels,Nott'm Forest,Goalkeeper,5.1,4.906542,139,27.254902,3.8,0.000693,-0.001901,2.4,"GW33:TOT(A)(3), GW34:BRE(H)(2), GW35:CRY(A)(3)..."
8,Mbeumo,Brentford,Midfielder,8.0,4.888643,192,24.000000,4.8,0.609168,0.005893,3.0,"GW33:BHA(H)(3), GW34:NFO(A)(4), GW35:MUN(H)(3)..."
9,Pickford,Everton,Goalkeeper,5.1,4.880568,131,25.686275,4.5,0.008754,0.000000,3.0,"GW33:MCI(H)(4), GW34:CHE(A)(4), GW35:IPS(H)(2)..."
